## Download datasets

In [2]:
import nltk
from simpletransformers.classification import ClassificationModel

nltk.download('punkt')
model = ClassificationModel(
    "roberta", "roberta-base", use_cuda=False, args={"output_dir": "model-outputs"}
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Construct training data

In [4]:
import pandas as pd

def get_class_data(file, label):
    with open(file) as f:
        text = f.read()
        sentences = nltk.tokenize.sent_tokenize(text)
        return [[s, label] for s in sentences]

positive_training_file = './data/training/global-warming.txt'
negative_training_file = './data/training/finest-hour.txt'

positive_training_data = get_class_data(positive_training_file, 1)
negative_training_data = get_class_data(negative_training_file, 0)
training_data = positive_training_data + negative_training_data


training_df = pd.DataFrame(training_data)
training_df.columns = ["text", "labels"]

In [5]:
training_df.head()

,text,labels
0,Global warming is the ongoing rise of the aver...,1
1,"It is a major aspect of climate change which, ...",1
2,While there have been prehistoric periods of g...,1
3,The Intergovernmental Panel on Climate Change ...,1
4,These findings have been recognized by the nat...,1


## Train model

In [6]:
model.train_model(training_df, args={"overwrite_output_dir": True, "train_batch_size": 1})

Running loss: 0.000305

/opt/conda/envs/transformers/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.000085


/opt/conda/envs/transformers/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


## Evaluate model

In [8]:
eval_data = [
    ["The remainder of the additional energy has melted ice and warmed the continents and the atmosphere.", 1],
    ["Another line of evidence for the warming not being due to the Sun is how temperature changes differ at different levels in the Earth's atmosphere.", 1],
    ["We shall fight in France, we shall fight on the seas and oceans, we shall fight with growing confidence and growing strength in the air, we shall defend our island, whatever the cost may be.", 0],
    ["The British Empire and the French Republic, linked together in their cause and in their need, will defend to the death their native soil, aiding each other like good comrades to the utmost of their strength.", 0]
    
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

result, model_outputs, wrong_predictions = model.eval_model(eval_df)

result, wrong_predictions

({'mcc': 1.0,
  'tp': 2,
  'tn': 2,
  'fp': 0,
  'fn': 0,
  'eval_loss': 0.00042187204235233366},
 [])